In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences,to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Input, InputLayer, Embedding, TimeDistributed,Bidirectional
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords,brown, treebank
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.metrics import accuracy_score,log_loss

In [3]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [4]:


import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [5]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [6]:
d1=brown.tagged_sents(tagset='universal')

In [7]:
len(d1)

57340

In [8]:

d2=treebank.tagged_sents(tagset='universal')

In [9]:
len(d2)

3914

In [10]:
data=d1+d2

In [11]:
len(data)

61254

In [12]:
x=[]
y=[]

for i in data:
    x1=[]
    y1=[]
    for j in i:
        x1.append(j[0])
        y1.append(j[1])
    x.append(x1)
    y.append(y1)

In [13]:
len(x)

61254

In [14]:
len(y)

61254

In [15]:


c=0

for i in range(len(x)):
    if(len(x[i])==len(y[i])):
        pass
    else:
        c+=1

print(c)


0


In [16]:
tk_x=Tokenizer(oov_token='UNK')
tk_x.fit_on_texts(x)

tk_y=Tokenizer(oov_token='oov',)
tk_y.fit_on_texts(y)

In [17]:
len(tk_x.word_index)

53233

In [18]:
len(tk_y.word_index)

13

In [19]:
tk_y.word_index

{'oov': 1,
 'noun': 2,
 'verb': 3,
 '.': 4,
 'adp': 5,
 'det': 6,
 'adj': 7,
 'adv': 8,
 'pron': 9,
 'conj': 10,
 'prt': 11,
 'num': 12,
 'x': 13}

In [20]:
tk_x.word_index

{'UNK': 1,
 'the': 2,
 ',': 3,
 '.': 4,
 'of': 5,
 'and': 6,
 'to': 7,
 'a': 8,
 'in': 9,
 'that': 10,
 'is': 11,
 'for': 12,
 'was': 13,
 'he': 14,
 '``': 15,
 "''": 16,
 'it': 17,
 'with': 18,
 'as': 19,
 'on': 20,
 'his': 21,
 'be': 22,
 'at': 23,
 'by': 24,
 ';': 25,
 'this': 26,
 'had': 27,
 'i': 28,
 'are': 29,
 'from': 30,
 'not': 31,
 '?': 32,
 'but': 33,
 'or': 34,
 'have': 35,
 'an': 36,
 'they': 37,
 'which': 38,
 '--': 39,
 'were': 40,
 'one': 41,
 'you': 42,
 'all': 43,
 'her': 44,
 'she': 45,
 'would': 46,
 'their': 47,
 'there': 48,
 'has': 49,
 'we': 50,
 'him': 51,
 'been': 52,
 'said': 53,
 'will': 54,
 ')': 55,
 '(': 56,
 'when': 57,
 'more': 58,
 'who': 59,
 'if': 60,
 'no': 61,
 'its': 62,
 'out': 63,
 'so': 64,
 'up': 65,
 'about': 66,
 'what': 67,
 'than': 68,
 'new': 69,
 ':': 70,
 'into': 71,
 'can': 72,
 'them': 73,
 'other': 74,
 'only': 75,
 'some': 76,
 'could': 77,
 'time': 78,
 'these': 79,
 '!': 80,
 'two': 81,
 'may': 82,
 'first': 83,
 'any': 84,
 'do'

In [21]:
x_data=tk_x.texts_to_sequences(x)
y_data=tk_y.texts_to_sequences(y)

In [22]:
len(x_data)

61254

In [23]:
final_x=pad_sequences(x_data,padding='post',value=0)
final_y=pad_sequences(y_data,padding='post',value=0)

In [24]:
final_x.shape

(61254, 271)

In [25]:
final_y.shape

(61254, 271)

In [26]:
final_y=to_categorical(final_y)

In [27]:
final_y.shape

(61254, 271, 14)

In [28]:
x_train,x_test,y_train,y_test=train_test_split(final_x,final_y,test_size=0.2,random_state=1)

In [29]:
x_train.shape

(49003, 271)

In [30]:
y_train.shape

(49003, 271, 14)

In [31]:
x_test.shape

(12251, 271)

In [32]:
y_test.shape

(12251, 271, 14)

In [34]:
model=Sequential()
model.add(InputLayer((271,)))
model.add(Embedding(53233+1,5))
model.add(Bidirectional(GRU(units=30,return_sequences=True)))
model.add(TimeDistributed(Dense(units=14, activation='softmax')))

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 271, 5)            266170    
                                                                 
 bidirectional (Bidirection  (None, 271, 60)           6660      
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 271, 14)           854       
 ributed)                                                        
                                                                 
Total params: 273684 (1.04 MB)
Trainable params: 273684 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [37]:
model.fit(x_train,y_train,batch_size=120,epochs=15,validation_split=0.2)

Epoch 1/15
327/327 [==============================] - 44s 114ms/step - loss: 0.4264 - accuracy: 0.9242 - val_loss: 0.1882 - val_accuracy: 0.9382
Epoch 2/15
327/327 [==============================] - 16s 47ms/step - loss: 0.1645 - accuracy: 0.9456 - val_loss: 0.1445 - val_accuracy: 0.9566
Epoch 3/15
327/327 [==============================] - 13s 39ms/step - loss: 0.1051 - accuracy: 0.9708 - val_loss: 0.0672 - val_accuracy: 0.9796
Epoch 4/15
327/327 [==============================] - 10s 30ms/step - loss: 0.0451 - accuracy: 0.9888 - val_loss: 0.0310 - val_accuracy: 0.9925
Epoch 5/15
327/327 [==============================] - 9s 29ms/step - loss: 0.0236 - accuracy: 0.9943 - val_loss: 0.0200 - val_accuracy: 0.9950
Epoch 6/15
327/327 [==============================] - 8s 26ms/step - loss: 0.0158 - accuracy: 0.9962 - val_loss: 0.0154 - val_accuracy: 0.9960
Epoch 7/15
327/327 [==============================] - 9s 27ms/step - loss: 0.0120 - accuracy: 0.9969 - val_loss: 0.0131 - val_accuracy: 0

In [38]:
x_test[1]

array([42477,  1939, 15131, 42478,     4,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [39]:
y_pred=np.argmax(model.predict(x_test),axis=1)

383/383 [==============================] - 4s 8ms/step


In [40]:
test=['Hi hello, how are you']

In [41]:
tk_x.texts_to_sequences(test)

[[12576, 8758, 127, 29, 42]]

In [42]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(test))

['hi hello how are you']

In [43]:
test1=pad_sequences(tk_x.texts_to_sequences(test),maxlen=271,padding='post',value=0)

In [44]:
np.argmax(model.predict(test1)[0],axis=1)[np.argmax(model.predict(test1)[0],axis=1)!=0]

1/1 [==============================] - 0s 23ms/step


array([11, 11,  8,  3,  9])

In [45]:
tk_y.sequences_to_texts([[11, 11,  8,  3,  9]])

['prt prt adv verb pron']

In [46]:
pickle.dump(tk_x,open(r"C:\Users\HP\Desktop\pos\tk_x.pkl",'wb'))

In [47]:
pickle.dump(tk_y,open(r'C:\Users\HP\Desktop\pos\tk_y.pkl','wb'))
pickle.dump(model,open(r"C:\Users\HP\Desktop\pos\model.pkl",'wb'))

In [49]:
x="My name is Kishore"
x=x.split()

In [50]:
x

['My', 'name', 'is', 'Kishore']

In [52]:
model=pickle.load(open(r"/content/C:\Users\HP\Desktop\pos\model.pkl","rb"))
tk_x=pickle.load(open(r"/content/C:\Users\HP\Desktop\pos\tk_x.pkl","rb"))
tk_y=pickle.load(open(r"/content/C:\Users\HP\Desktop\pos\tk_y.pkl","rb"))

In [53]:
test=['Hi hello, how are you']


In [54]:
tk_x.texts_to_sequences(test)

[[12576, 8758, 127, 29, 42]]

In [55]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(test))

['hi hello how are you']

In [56]:
test1=pad_sequences(tk_x.texts_to_sequences(test),maxlen=271,padding='post',value=0)

In [57]:
np.argmax(model.predict(test1)[0],axis=1)[np.argmax(model.predict(test1)[0],axis=1)!=0]

1/1 [==============================] - 0s 23ms/step


array([11, 11,  8,  3,  9])

In [58]:
tk_y.sequences_to_texts([[11, 11,  8,  3,  9]])

['prt prt adv verb pron']